## 라이브러리 불러오기

In [2]:
import pandas as pd
import numpy as np

import requests
import time
import openpyxl
from sklearn.metrics.pairwise import cosine_similarity

import ssl
from urllib.request import urlopen
import xml.etree.ElementTree as ET

import warnings
warnings.filterwarnings('ignore')

# ① 국가별 대한(對韓) 수입 품목별 비중 기반의 국가 간 유사도 계산

In [8]:
#관세청_품목별 국가별 수출입실적(GW) 데이터_2022 불러오기
df_2022_raw = pd.read_csv('country_2022_finn.csv')
df_2022_raw

,Unnamed: 0,balPayments,expDlr,expWgt,hsCd,impDlr,impWgt,statCd,statCdCntnKor1,statKor,year
0,0,538216,705398,5890,-,167182,19333,-,-,-,총계
1,1,-318,0,0,4202211090,318,1,AD,안도라,기타,2022.01
2,2,-540,0,0,4202999000,540,2,AD,안도라,기타,2022.01
3,3,-3724,0,0,6110909000,3724,15,AD,안도라,기타,2022.01
4,4,-231,0,0,7117199000,231,1,AD,안도라,기타,2022.01
...,...,...,...,...,...,...,...,...,...,...,...
1576723,73330,6350,6350,60,9018399000,0,0,ZW,짐바브웨,부분품과 부속품,2022.12
1576724,73331,1700,1700,3,9022909010,0,0,ZW,짐바브웨,엑스선을 사용하는 기기의 것,2022.12
1576725,73332,20,20,1,9022909090,0,0,ZW,짐바브웨,기타,2022.12
1576726,73333,162,162,1,9025809000,0,0,ZW,짐바브웨,기타,2022.12


In [9]:
# 데이터 확인
df_2022_raw.head(5)

,Unnamed: 0,balPayments,expDlr,expWgt,hsCd,impDlr,impWgt,statCd,statCdCntnKor1,statKor,year
0,0,538216,705398,5890,-,167182,19333,-,-,-,총계
1,1,-318,0,0,4202211090,318,1,AD,안도라,기타,2022.01
2,2,-540,0,0,4202999000,540,2,AD,안도라,기타,2022.01
3,3,-3724,0,0,6110909000,3724,15,AD,안도라,기타,2022.01
4,4,-231,0,0,7117199000,231,1,AD,안도라,기타,2022.01


In [10]:
# 'Unnamed: 0' 컬럼 제거
df_2022_raw.drop('Unnamed: 0', axis=1, inplace=True)

## 데이터 전처리

In [11]:
# 총계 행 제거
df_2022_raw = df_2022_raw.loc[df_2022_raw['year'] != '총계', :]

# 음수인 수출액 제거
df_2022_raw = df_2022_raw.loc[df_2022_raw['expWgt'] >= 0]

# 날짜 타입 통일 및 오류 수정
# 오류) 10월 - 자료형을 float로 인식해 2022.1로 표기됨을 확인
df_2022_raw.loc[df_2022_raw['year'] == '2022.1', 'year'] = '2022.01'
df_2022_raw.loc[df_2022_raw['year'] == 2022.1, 'year'] = '2022.10'
df_2022_raw['year'] = df_2022_raw['year'].astype(str)
df_2022_raw.sort_values(by=['year', 'statCd', 'hsCd'], axis=0)

# hscode 앞자리가 0이라 9자리로 표시되는 것을 앞에 0 붙이기
df_2022_raw['hsCd'] = df_2022_raw['hsCd'].astype(str)
df_2022_raw.loc[df_2022_raw['hsCd'].str.len() == 9, 'hsCd'] = df_2022_raw.loc[df_2022_raw['hsCd'].str.len() == 9, 'hsCd'].str.zfill(10)

# 인덱스 초기화
df_2022_raw = df_2022_raw.reset_index(drop=True)

# hsCode 6자리 사용
df_2022_raw['hsCd'] = df_2022_raw['hsCd'].str[:6]

# 결측치로 잘못 인식된 국가코드 나미비아 'NA'(str)로 바꾸기
df_2022_raw['statCd'] = df_2022_raw['statCd'].fillna('NA')

In [12]:
# 전처리 후 csv파일로 저장
df_2022_raw.to_csv('df_2022.csv', index=False)

## 국가, 품목 피봇 생성

In [13]:
# 불러오기
df_2022 = pd.read_csv('df_2022.csv')

In [14]:
df_2022.shape

(1576484, 10)

In [15]:
#'statCd'와 'hsCd' 열을 기준으로 그룹화 후, 'expDlr' 합계 그룹화
group_df = df_2022.groupby(['statCd','hsCd'], as_index=False)[['expDlr']].sum()
group_df

,statCd,hsCd,expDlr
0,AD,300490,40
1,AD,330499,10981
2,AD,330790,65
3,AD,391910,50
4,AD,392010,963
...,...,...,...
193320,ZW,950662,2170
193321,ZW,960390,0
193322,ZW,960990,415
193323,ZW,970191,0


In [16]:
# 국가간 유사도를 만들기 위한 pivot table 구성
pv = group_df.pivot_table(index='statCd',columns='hsCd',values='expDlr')

#확인
pv

hsCd,10121,10129,10190,10221,10310,10391,10392,10410,10420,10511,...,970510,970521,970522,970529,970531,970539,970600,970610,970690,999999
statCd,,,,,,,,,,,,,,,,,,,,,
AD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AE,NaN,107395.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68128.0,0.0
AF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
YT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0


In [22]:
# 비중을 구하기 위해 국가별 수출액 합계
pv['total'] = pv.sum(axis=1)

# 국가별 각 상품의 수출액 확인
pv.head()

hsCd,10121,10129,10190,10221,10310,10391,10392,10410,10420,10511,...,970521,970522,970529,970531,970539,970600,970610,970690,999999,total
statCd,,,,,,,,,,,,,,,,,,,,,
AD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,705398.0000000000
AE,NaN,107395.0000000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68128.0000000000,0.0000000000,3637560372.0000000000
AF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19138657.0000000000
AG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3456986.0000000000
AI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,169051.0000000000


## 비중 구하기

In [23]:
# 국가별 각 품목의 수출 비중 구하기
for column in pv.columns:
    pv[column] = pv[column] / pv['total']

In [24]:
# total 컬럼 제거
pv.drop('total', axis=1, inplace=True)

# 확인
pv

hsCd,10121,10129,10190,10221,10310,10391,10392,10410,10420,10511,...,970510,970521,970522,970529,970531,970539,970600,970610,970690,999999
statCd,,,,,,,,,,,,,,,,,,,,,
AD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AE,NaN,0.0000295239,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000187290,0.0000000000
AF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
YT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0000000000,NaN,NaN,NaN,0.0000000000


## ② 코사인 유사도를 활용한 유망 품목 도출

In [25]:
# 코사인 유사도를 활용한 국가 간의 유사도
matrix_dummy = pv.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=pv.index, columns=pv.index)

In [26]:
# 한국에 대한 국가 간 품목 수입 유사도
user_similarity

statCd,AD,AE,AF,AG,AI,AL,AM,AO,AQ,AR,...,VI,VN,VU,WF,WS,YE,YT,ZA,ZM,ZW
statCd,,,,,,,,,,,,,,,,,,,,,
AD,1.0000000000,0.0161496222,0.0001867247,0.0357823254,0.0117658804,0.0283489080,0.0248094309,0.0121461807,0.0000027649,0.0008990253,...,0.0001270136,0.0123016154,0.0094608899,0.0000000000,0.0004545594,0.0090416350,0.0000000000,0.0080787410,0.0002841116,0.0020724474
AE,0.0161496222,1.0000000000,0.0268652779,0.3620812228,0.2107477662,0.3091188213,0.4424899741,0.1826798463,0.0063263058,0.3648226231,...,0.1209857834,0.1264549516,0.3275921005,0.0934769397,0.0460721688,0.3801937257,0.0112538819,0.4641893378,0.1177906693,0.0528819259
AF,0.0001867247,0.0268652779,1.0000000000,0.0073093881,0.0278814032,0.0177515713,0.0107997177,0.0757490136,0.0010806016,0.0210899506,...,0.0000056589,0.0051846014,0.0074579537,0.0035483437,0.0009369542,0.0815672260,0.0000000000,0.0127590081,0.0446784896,0.0313523355
AG,0.0357823254,0.3620812228,0.0073093881,1.0000000000,0.3272161138,0.6547906588,0.5346811234,0.3246487867,0.0000275373,0.0192203106,...,0.0020131783,0.0202108102,0.3039488151,0.0332371685,0.0536484240,0.2782670161,0.0059054863,0.2206893157,0.0406397397,0.0703366588
AI,0.0117658804,0.2107477662,0.0278814032,0.3272161138,1.0000000000,0.4505882529,0.2751828040,0.2811162583,0.0111066866,0.0217383720,...,0.0261818850,0.0197447022,0.6568500936,0.4373291192,0.0099444250,0.1509570105,0.0056018041,0.3883429923,0.0553154213,0.1208956747
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YE,0.0090416350,0.3801937257,0.0815672260,0.2782670161,0.1509570105,0.2356195838,0.3010035867,0.1553416178,0.0012570351,0.0707527341,...,0.0315075000,0.0274718423,0.2259564369,0.2341607978,0.0058616126,1.0000000000,0.2107716944,0.2619435003,0.1226152277,0.0618674488
YT,0.0000000000,0.0112538819,0.0000000000,0.0059054863,0.0056018041,0.0292695683,0.0387030401,0.1407042165,0.0000000000,0.0004544565,...,0.0000000000,0.0005449745,0.1979192556,0.5899637532,0.0026551888,0.2107716944,1.0000000000,0.1657580780,0.0013507158,0.0000000000
ZA,0.0080787410,0.4641893378,0.0127590081,0.2206893157,0.3883429923,0.2617311945,0.3655237591,0.2628771228,0.0052806437,0.4922248613,...,0.0292099221,0.1794680576,0.5457482704,0.4368375689,0.0320339767,0.2619435003,0.1657580780,1.0000000000,0.0953792219,0.0771380263


In [27]:
# 소수점 10자리까지 확인하기 위한 포맷
pd.options.display.float_format = '{:.10f}'.format

In [28]:
# 유사도, 품목별 수입 예상 비중 함수
def calculate_hssi(country_code):
    stat = user_similarity[country_code].sort_values().tail(11)         # 한국에 대한 선택 국가의 수출과 가장 유사한 10개의 국가
    stat2 = list(stat.index)
    stat3 = pv.loc[stat2]                                               # 선택 국가와 수출이 유사한 10개의 나라별 상품 수출 비중

    for col in list(stat3.columns):
        if np.isnan(stat3.loc[country_code, col]) != True:              # 선택 국가의 행이 NaN인 열만 유지
            stat3.drop(col, axis=1, inplace=True)

    col = stat3.isna().sum().sort_values(ascending=True).head(30).index # 결측치가 가장 적은 상위 상품
    table = stat3[col]                                                  # 한 나라당 교역 비중
    sim = user_similarity.loc[stat2][[country_code]]                    # 선택 국가와 10개 국가 간의 유사도

    hssi = pd.DataFrame()

    for col in table.columns.values:
        hs = (table[col] * sim[country_code]) / (sim[country_code].sum() - 1) # (비중 * 유사도) / 유사도의 합
        hssi[col] = hs

    result = hssi.sum().sort_values(ascending=False)                    # 가중평균한 것의 각 행씩 합: 품목별 수입 예상 비중
    return sim, sim.index.values, result, result.index

### 선진국 - 호주(AU), 스페인(ES), 뉴질랜드(NZ)

In [29]:
# 호주
calculate_hssi('AU')

(statCd           AU
 statCd             
 NZ     0.9607390169
 CM     0.9618846128
 BD     0.9636762696
 EH     0.9639654366
 CF     0.9639787398
 TG     0.9640831726
 PF     0.9648346155
 PW     0.9761138200
 GU     0.9827286200
 FJ     0.9845582062
 AU     1.0000000000,
 array(['NZ', 'CM', 'BD', 'EH', 'CF', 'TG', 'PF', 'PW', 'GU', 'FJ', 'AU'],
       dtype=object),
 550130   0.0007341043
 310230   0.0006420138
 310530   0.0003610791
 730120   0.0003506682
 721011   0.0002079833
 721699   0.0001518960
 411420   0.0000885131
 870210   0.0000812505
 551513   0.0000405132
 844839   0.0000276859
 701914   0.0000162481
 741129   0.0000142448
 252390   0.0000067367
 848010   0.0000044827
 722240   0.0000044587
 848020   0.0000035739
 291090   0.0000023083
 30732    0.0000022807
 722230   0.0000020301
 850212   0.0000020014
 831190   0.0000015846
 853210   0.0000011105
 846890   0.0000008691
 283329   0.0000008675
 630539   0.0000007893
 844520   0.0000001521
 821194   0.0000000790
 847740 

In [30]:
# 스페인
calculate_hssi('ES')

(statCd           ES
 statCd             
 PS     0.5511525375
 MU     0.5639547026
 RE     0.5768810744
 IT     0.6061791948
 GP     0.6064818564
 GB     0.6231891295
 MQ     0.6254005514
 YT     0.7200724171
 GF     0.7202293561
 GI     0.7762234244
 ES     1.0000000000,
 array(['PS', 'MU', 'RE', 'IT', 'GP', 'GB', 'MQ', 'YT', 'GF', 'GI', 'ES'],
       dtype=object),
 842952   0.0148493550
 842951   0.0011045544
 300241   0.0007717664
 842720   0.0007371878
 240220   0.0003106079
 390529   0.0001849025
 870600   0.0001506774
 870333   0.0001063521
 843229   0.0000271466
 850660   0.0000042234
 902230   0.0000027190
 960860   0.0000024625
 842381   0.0000024362
 320413   0.0000022424
 821510   0.0000006300
 630110   0.0000004200
 620891   0.0000003604
 810196   0.0000001099
 90510    0.0000001042
 821210   0.0000001019
 620819   0.0000000717
 620722   0.0000000112
 620423   0.0000000079
 620829   0.0000000043
 620429   0.0000000018
 621230   0.0000000009
 271099   0.0000000000
 630312 

In [31]:
# 뉴질랜드
calculate_hssi('NZ')

(statCd           NZ
 statCd             
 CM     0.9009258420
 EH     0.9048458741
 CF     0.9048498958
 TG     0.9049030995
 BD     0.9095500692
 EC     0.9418890142
 GU     0.9574021576
 AU     0.9607390169
 FJ     0.9745365605
 PW     0.9776174382
 NZ     1.0000000000,
 array(['CM', 'EH', 'CF', 'TG', 'BD', 'EC', 'GU', 'AU', 'FJ', 'PW', 'NZ'],
       dtype=object),
 300241   0.0072574447
 740811   0.0029585276
 550130   0.0009141692
 870210   0.0006108558
 382211   0.0003238370
 841510   0.0003033589
 730490   0.0002609477
 870880   0.0002542110
 845530   0.0000835989
 841829   0.0000405575
 841830   0.0000298105
 320619   0.0000256505
 382000   0.0000176597
 293339   0.0000161192
 940210   0.0000123311
 381900   0.0000122652
 730640   0.0000101630
 844859   0.0000091025
 851650   0.0000064749
 730711   0.0000057886
 370110   0.0000055942
 844331   0.0000048617
 842441   0.0000040918
 381090   0.0000031449
 401190   0.0000030965
 731290   0.0000023402
 830242   0.0000013392
 271020 

### 신흥시장국 - 베네주엘라(VE), 그리스(GR), 필리핀(PH)

In [32]:
# 베네주엘라
calculate_hssi('VE')

(statCd           VE
 statCd             
 CW     0.3085594286
 PR     0.3112653214
 VG     0.3130649598
 KZ     0.3135335620
 BO     0.3145681786
 SA     0.3275969953
 GA     0.3287464133
 LY     0.3321449347
 CA     0.3328437431
 PE     0.4132473824
 VE     1.0000000000,
 array(['CW', 'PR', 'VG', 'KZ', 'BO', 'SA', 'GA', 'LY', 'CA', 'PE', 'VE'],
       dtype=object),
 870322   0.0297693068
 870332   0.0230125709
 842952   0.0157695988
 870210   0.0075947959
 390210   0.0061990459
 870340   0.0049452155
 190230   0.0012517658
 842139   0.0009788069
 842131   0.0009635961
 190590   0.0007326367
 210690   0.0005663572
 650500   0.0001719494
 903190   0.0001168595
 850720   0.0001072627
 902790   0.0000979169
 711790   0.0000671926
 401032   0.0000670009
 847340   0.0000648080
 843120   0.0000572224
 842119   0.0000475734
 950300   0.0000459207
 640419   0.0000429307
 848250   0.0000266504
 491199   0.0000247528
 620449   0.0000167107
 420212   0.0000113945
 851310   0.0000063007
 731990 

In [33]:
#그리스
calculate_hssi('GR')

(statCd           GR
 statCd             
 SA     0.3583765549
 MT     0.5108822363
 BS     0.5499373261
 LR     0.6289775339
 NO     0.7175622501
 CY     0.7495177368
 PA     0.7828831126
 MH     0.9853466115
 IM     0.9919044457
 BM     0.9922162570
 GR     1.0000000000,
 array(['SA', 'MT', 'BS', 'LR', 'NO', 'CY', 'PA', 'MH', 'IM', 'BM', 'GR'],
       dtype=object),
 870421   0.0006548159
 841510   0.0005100700
 841950   0.0003043233
 870422   0.0002261602
 841430   0.0002015996
 300450   0.0001381303
 761510   0.0000950146
 854239   0.0000740570
 300241   0.0000626911
 830300   0.0000605257
 901580   0.0000450664
 854420   0.0000199403
 841582   0.0000118331
 401033   0.0000053497
 851660   0.0000042716
 30389    0.0000028446
 845011   0.0000027042
 730792   0.0000017950
 846721   0.0000012428
 854800   0.0000004674
 950450   0.0000003083
 210610   0.0000002567
 850519   0.0000001113
 392640   0.0000000666
 400821   0.0000000454
 741533   0.0000000419
 340540   0.0000000147
 731029 

In [34]:
# 필리핀
calculate_hssi('PH')

(statCd           PH
 statCd             
 SG     0.6804600286
 AU     0.7064041347
 GU     0.7109062132
 EC     0.7291112452
 FJ     0.7291135960
 PW     0.7339056042
 JP     0.7396288691
 NZ     0.7527649695
 MY     0.7646983502
 ID     0.7953829331
 PH     1.0000000000,
 array(['SG', 'AU', 'GU', 'EC', 'FJ', 'PW', 'JP', 'NZ', 'MY', 'ID', 'PH'],
       dtype=object),
 870321   0.0049616519
 721061   0.0023682133
 720839   0.0018880188
 848610   0.0000844301
 401694   0.0000776523
 540419   0.0000245386
 741122   0.0000243111
 320419   0.0000166740
 790500   0.0000135850
 690100   0.0000120922
 440799   0.0000116529
 630293   0.0000079456
 60319    0.0000034620
 851020   0.0000033580
 701710   0.0000028202
 901720   0.0000026014
 871420   0.0000023944
 845620   0.0000023525
 480439   0.0000022692
 520942   0.0000017660
 830150   0.0000014952
 732429   0.0000012284
 160569   0.0000010030
 190420   0.0000007272
 640192   0.0000006654
 842122   0.0000006002
 820150   0.0000002643
 551349 

### 저개발국 - 네팔(NP), 우루과이(UY), 가봉(GA), 볼리비아(BO)

In [35]:
calculate_hssi('NP')

(statCd           NP
 statCd             
 DE     0.5126292781
 DK     0.5260504788
 MU     0.5562192482
 BT     0.6088301846
 NO     0.6384117030
 GB     0.7051321229
 RE     0.7287911816
 FI     0.7497030713
 SE     0.7965417230
 IS     0.8668929150
 NP     1.0000000000,
 array(['DE', 'DK', 'MU', 'BT', 'NO', 'GB', 'RE', 'FI', 'SE', 'IS', 'NP'],
       dtype=object),
 870321   0.0531001054
 870322   0.0153714058
 271019   0.0082949512
 401120   0.0077388774
 852589   0.0016797310
 848210   0.0008923566
 950790   0.0007592149
 950730   0.0006121917
 842139   0.0002256342
 852910   0.0002068282
 842489   0.0001558735
 560750   0.0000862690
 220210   0.0000800408
 392310   0.0000754828
 960810   0.0000666184
 830250   0.0000366141
 330430   0.0000262898
 392610   0.0000219982
 820590   0.0000205019
 620920   0.0000184662
 620463   0.0000117507
 620640   0.0000090057
 842549   0.0000070335
 960390   0.0000061910
 482090   0.0000055898
 950440   0.0000044404
 620452   0.0000041009
 610442 

In [36]:
calculate_hssi('UY')

(statCd           UY
 statCd             
 SY     0.3388815961
 TM     0.3485790951
 PK     0.3750405618
 DM     0.3763015273
 DZ     0.3822093503
 SD     0.4092730331
 AE     0.4371344561
 UZ     0.4419172121
 SK     0.4985203102
 HN     0.6687517707
 UY     1.0000000000,
 array(['SY', 'TM', 'PK', 'DM', 'DZ', 'SD', 'AE', 'UZ', 'SK', 'HN', 'UY'],
       dtype=object),
 840991   0.0066053886
 841480   0.0045750890
 848310   0.0037481489
 392051   0.0015897075
 340290   0.0013311398
 840734   0.0011142087
 901849   0.0010661831
 841950   0.0010264332
 732020   0.0008921281
 840820   0.0008442846
 870891   0.0007172446
 853620   0.0007057068
 841459   0.0006172034
 851230   0.0003584455
 400942   0.0001930238
 848140   0.0001885751
 731829   0.0001426634
 281122   0.0001195347
 842490   0.0000982268
 392410   0.0000645272
 830710   0.0000569554
 848320   0.0000542504
 851310   0.0000449008
 848280   0.0000443817
 848220   0.0000384420
 848250   0.0000299248
 848299   0.0000263054
 848360 

In [37]:
calculate_hssi('GA')

(statCd           GA
 statCd             
 TJ     0.9261857340
 TC     0.9266693718
 LY     0.9299658946
 CV     0.9307623620
 KY     0.9325654851
 MF     0.9359129130
 CW     0.9384608800
 PR     0.9388433274
 VG     0.9561638466
 KZ     0.9568122995
 GA     1.0000000000,
 array(['TJ', 'TC', 'LY', 'CV', 'KY', 'MF', 'CW', 'PR', 'VG', 'KZ', 'GA'],
       dtype=object),
 870380   0.0154699979
 870340   0.0102056335
 870422   0.0065986030
 300241   0.0035979491
 870830   0.0026554052
 220299   0.0019781705
 841391   0.0014377627
 210690   0.0012364070
 902789   0.0011164054
 870880   0.0006364174
 853710   0.0005295373
 847989   0.0004948957
 901812   0.0004735757
 870829   0.0003860196
 560900   0.0003826374
 843141   0.0003799576
 870894   0.0003285715
 392190   0.0002527332
 330510   0.0002195856
 842121   0.0001629915
 330790   0.0001223721
 870891   0.0001000567
 841381   0.0000592624
 340290   0.0000439181
 640419   0.0000234021
 852349   0.0000163118
 611090   0.0000108810
 630790 

In [38]:
calculate_hssi('BO')

(statCd           BO
 statCd             
 TC     0.8870858219
 LY     0.9012304885
 AG     0.9060903530
 KZ     0.9132993505
 MF     0.9233390047
 VG     0.9250247875
 PR     0.9263571935
 TJ     0.9274085828
 KY     0.9366158089
 CW     0.9388609053
 BO     1.0000000000,
 array(['TC', 'LY', 'AG', 'KZ', 'MF', 'VG', 'PR', 'TJ', 'KY', 'CW', 'BO'],
       dtype=object),
 870421   0.0457980265
 870422   0.0185494565
 850710   0.0173596915
 870380   0.0118979456
 271019   0.0062681082
 870210   0.0057943996
 401110   0.0030647448
 841510   0.0009492499
 841590   0.0006901520
 841810   0.0005095672
 940330   0.0003057933
 401220   0.0002916787
 330510   0.0002149656
 401699   0.0001289087
 330790   0.0001200273
 845020   0.0001059454
 870891   0.0001006219
 340130   0.0000776334
 901832   0.0000618000
 903289   0.0000471978
 540769   0.0000467090
 340290   0.0000434213
 870810   0.0000339292
 640419   0.0000231395
 841350   0.0000212149
 841381   0.0000167647
 730799   0.0000090836
 853669 

# ③ 한국 특정 품목의 수출 경쟁력 검증 (RCA)

## RCA 분자

## HS CODE 10단위 관세청 품목별 수출입 실적 데이터_2022 불러오기

## 데이터 전처리

In [39]:
item_raw = pd.read_csv('item_raw.csv')

In [40]:
# 총계 행 제거
item = item_raw.loc[item_raw['year'] != '총계', :]

# 음수인 수출액 제거
item['expWgt'] = item['expWgt'].astype(float)
item = item.loc[item['expWgt'] >= 0]

# 날짜 타입 통일 및 오류 수정
# 오류) 10월 - 자료형을 float로 인식해 2022.1로 표기됨을 확인
item.loc[item['year'] == 2021.1, 'year'] = '2021.10'
item.loc[item['year'] == 2022.1, 'year'] = '2022.10'
item['year'] = item['year'].astype(str)
item.sort_values(by=['year', 'hsCode'], axis=0)

# hscode 앞자리가 0이라 9자리로 표시되는 것을 앞에 0 붙이기
item['hsCode'] = item['hsCode'].astype(str)
item.loc[item['hsCode'].str.len() == 9, 'hsCode'] = item.loc[item['hsCode'].str.len() == 9, 'hsCode'].str.zfill(10)
item = item.reset_index(drop=True)
item['hsCode'] = item['hsCode'].str[:6]

In [41]:
item.to_csv('item.csv', index=False)

## 한국 i품목 총 수출액 (분자의 분자)

### 방법 1) 한 국가씩 i품목 총 수출액 계산
- VE, GR, PH

In [42]:
def k_item_x(hs):
    total = item.loc[item['hsCode'] == hs]['expDlr'].astype(int).sum()
    return total

In [43]:
# VE

numbers = ['870322', '870332', '842952', '870210', '390210', '870340', '190230', '842139', '842131', '190590',
           '210690', '650500', '903190', '850720', '902790', '711790', '401032', '847340', '843120', '842119',
           '950300', '640419', '848250', '491199', '620449', '420212', '851310', '731990', '482090', '731824']

for hs in numbers:
    result = k_item_x(hs)
    print(f'{hs}: {result}')

870322: 6266563064
870332: 3782770635
842952: 3235310598
870210: 457669787
390210: 2493172653
870340: 6263603912
190230: 862387282
842139: 790447993
842131: 81138263
190590: 329552401
210690: 730438126
650500: 59889706
903190: 486800364
850720: 68316420
902790: 78549761
711790: 10459596
401032: 8063972
847340: 74041804
843120: 159295037
842119: 9507225
950300: 80591339
640419: 74943187
848250: 23272736
491199: 41408425
620449: 5129226
420212: 14302426
851310: 9298578
731990: 3256811
482090: 5082948
731824: 9126911


In [44]:
# GR

numbers = ['870421', '841510', '841950', '870422', '841430', '300450', '761510', '854239', '300241', '830300',
           '901580', '854420', '841582', '401033', '851660', '030389', '845011', '730792', '846721', '854800',
           '950450', '210610', '850519', '392640', '400821', '741533', '340540', '731029', '890710', '271099']

for hs in numbers:
    result = k_item_x(hs)
    print(f'{hs}: {result}')

870421: 1376773738
841510: 538849889
841950: 516991509
870422: 158391695
841430: 1150558900
300450: 27089724
761510: 124087859
854239: 11628361856
300241: 941123395
830300: 49764096
901580: 23034425
854420: 140202730
841582: 61787312
401033: 4297037
851660: 322125503
030389: 82230503
845011: 14276833
730792: 89295003
846721: 1920199
854800: 104144960
950450: 24732174
210610: 13838373
850519: 94777530
392640: 11592317
400821: 25033607
741533: 10304060
340540: 79801112
731029: 87232274
890710: 1194736
271099: 8923


In [45]:
# PH

numbers = ['870321', '721061', '720839', '848610', '401694', '540419', '741122', '320419', '790500', '690100',
           '440799', '630293', '060319', '851020', '701710', '901720', '871420', '845620', '480439', '520942',
           '830150', '732429', '160569', '190420', '640192', '842122', '820150', '551349', '441911', '090620']

for hs in numbers:
    result = k_item_x(hs)
    print(f'{hs}: {result}')

870321: 1829455897
721061: 551339668
720839: 1093072742
848610: 216001921
401694: 6993740
540419: 45494597
741122: 33306501
320419: 11232907
790500: 4690866
690100: 4416629
440799: 1270505
630293: 4971482
060319: 291777
851020: 984624
701710: 1261307
901720: 1515882
871420: 592008
845620: 1612704
480439: 6365738
520942: 2663970
830150: 1649497
732429: 820655
160569: 420855
190420: 284249
640192: 548731
842122: 612257
820150: 343747
551349: 83314
441911: 86557
090620: 17858


In [46]:
# NP

numbers = ['870321', '870322', '271019', '401120', '852589', '848210', '950790', '950730', '842139', '852910',
           '842489', '560750', '220210', '392310', '960810', '830250', '330430', '392610', '820590', '620920',
           '620463', '620640', '842549', '960390', '482090', '950440', '620452', '610442', '611420', '400821']


for hs in numbers:
    result = k_item_x(hs)
    print(f'{hs}: {result}')

870321: 1829455897
870322: 6266563064
271019: 46318725022
401120: 926614789
852589: 2716386822
848210: 463592712
950790: 32300205
950730: 52785023
842139: 790447993
852910: 144857235
842489: 270764597
560750: 49967539
220210: 146648920
392310: 628504201
960810: 32453522
830250: 23202443
330430: 39335903
392610: 11635435
820590: 8766722
620920: 2976681
620463: 18933427
620640: 18938924
842549: 6775117
960390: 6398101
482090: 5082948
950440: 10973052
620452: 6566874
610442: 6398092
611420: 3958405
400821: 25033607


In [47]:
# UY

numbers = ['840991', '841480', '848310', '392051', '340290', '840734', '901849', '841950', '732020', '840820',
           '870891', '853620', '841459', '851230', '400942', '848140', '731829', '281122', '842490', '392410',
           '830710', '848320', '851310', '848280', '848220', '848250', '848299', '848360', '830241', '640399']

for hs in numbers:
    result = k_item_x(hs)
    print(f'{hs}: {result}')

840991: 1239752841
841480: 320493251
848310: 462426927
392051: 310000084
340290: 166011541
840734: 924480919
901849: 375322243
841950: 516991509
732020: 53866811
840820: 135730208
870891: 67400423
853620: 177674548
841459: 237347908
851230: 51455787
400942: 23699234
848140: 50162303
731829: 62275761
281122: 100640270
842490: 73031065
392410: 64987787
830710: 79660324
848320: 8831324
851310: 9298578
848280: 23553163
848220: 180173190
848250: 23272736
848299: 170088377
848360: 31919547
830241: 13611791
640399: 48879749


In [48]:
# GA

numbers = ['870380', '870340', '870422', '300241', '870830', '220299', '841391', '210690', '902789', '870880',
           '853710', '847989', '901812', '870829', '560900', '843141', '870894', '392190', '330510', '842121',
           '330790', '870891', '841381', '340290', '640419', '852349', '611090', '630790', '650500', '902610']

for hs in numbers:
    result = k_item_x(hs)
    print(f'{hs}: {result}')

870380: 8175751051
870340: 6263603912
870422: 158391695
300241: 941123395
870830: 715031803
220299: 481657973
841391: 344950258
210690: 730438126
902789: 217339636
870880: 630188333
853710: 1387485850
847989: 3694290076
901812: 768684707
870829: 2398707417
560900: 11637503
843141: 137547222
870894: 852004558
392190: 641528633
330510: 146183502
842121: 871592348
330790: 611853643
870891: 67400423
841381: 81348255
340290: 166011541
640419: 74943187
852349: 709249013
611090: 11884018
630790: 249202827
650500: 59889706
902610: 55607638


In [49]:
# BO

numbers = ['870421', '870422', '850710', '870380', '271019', '870210', '401110', '841510', '841590', '841810',
           '940330', '401220', '330510', '401699', '330790', '845020', '870891', '340130', '901832', '903289',
           '540769', '340290', '870810', '640419', '841350', '841381', '730799', '853669', '850811', '731816']

for hs in numbers:
    result = k_item_x(hs)
    print(f'{hs}: {result}')

870421: 1376773738
870422: 158391695
850710: 1801938738
870380: 8175751051
271019: 46318725022
870210: 457669787
401110: 2218555583
841510: 538849889
841590: 754239268
841810: 2183383108
940330: 11646314
401220: 9045076
330510: 146183502
401699: 294314821
330790: 611853643
845020: 534113635
870891: 67400423
340130: 248771488
901832: 136337455
903289: 724418772
540769: 114664813
340290: 166011541
870810: 55649482
640419: 74943187
841350: 71460093
841381: 81348255
730799: 219315741
853669: 1145428695
850811: 42964062
731816: 210941123


### 방법2) 한 번에 여러 국가의 i품목 총 수출액 계산
- AU, ES, NZ

In [50]:
# 호주(AU), 스페인(ES), 뉴질랜드(NZ)
numbers = ['550130','310230','310530','730120','721011','721699','411420','870210','551513','844839',
           '701914','741129','252390','848010','722240','848020','291090','030732','722230','850212',
           '831190','853210','846890','283329','630539','844520','821194','847740','091091','411310',
           '842952','842951','300241','842720','240220','390529','870600','870333','843229','850660',
           '902230','960860','842381','320413','821510','630110','620891','810196','090510','821210',
           '620819','620722','620423','620829','620429','621230','271099','630312','610729','170114',
           '300241','740811','550130','870210','382211','841510','730490','870880','845530','841829',
           '841830','320619','382000','293339','940210','381900','730640','844859','851650','730711',
           '370110','844331','842441','381090','401190','731290','830242','271020','901730','090111']

result = []

for hs in numbers:
    k_item_x = item.loc[(item['hsCode'] == hs)]
    result2 = k_item_x['expDlr'].sum()
    result.append(result2)
    print(result2)

85851824
99476172
22367830
1735535
3377611
3052378
33169224
457669787
2663281
19211734
698699
3723974
242880
15482272
988741
23015776
15091493
105421
4988414
16729221
5078909
13845203
18461496
8627553
1884788
3373271
5907586
79984246
127552
626659
3235310598
421494212
941123395
762713673
630781471
155754842
2664025
214420059
1782169
978570
12187484
3531006
13418909
2687909
220969
2116313
413661
3667766
6974
3874541
70097
230139
135455
2126161
20444
241914
8923
743227
1875
188854
941123395
222621747
85851824
457669787
72403349
538849889
70030232
630188333
101049516
1653827
3417676
19190285
12720142
119040082
7035454
9076002
198637155
16997894
168607
2137821
1044660
204328183
1501012
46331669
1943287
1677039
50652529
2864026
5945353
1396063


## 한국의 총수출액 (분자의 분모)

In [51]:
item['expDlr'].astype(float).sum()

681729790292.0

## RCA 분모

### i품목의 전세계 수출액 (분모의 분자)

### 방법 1) HSCODE 지정하여 해당품목 전세계 수출액 가져오기
- VE, GR, PH

In [52]:
# 해당품목 전세계 수출액

hs_code = "870322"  # hsCode 지정

# 지정한 hscode에 대한 url
url = f'https://www.kotra.or.kr/bigdata/visualization/global/search?baseYr=2022&expIsoWd2NatCd=ALL&impIsoWd2NatCd=ALL&hscd={hs_code}'
response = requests.get(url)
data = response.json()

if data["countryAmtTop5List"]:
    exp_amt = 0
    for item in data["expTrendList"]:   # 집계
        if (item["baseYr"] == "2022") and (item["expItcNatCd"] == "ALL"):  # 2022년 집계
            exp_amt = item["expAmt"]

    print("해당품목 전세계 수출액:", exp_amt)


해당품목 전세계 수출액: 96002635023


# ④ 특정 국가의 해당 품목 수입해당 품목 관련 규제,  수입국 및 수출국 확인

## 1. 규제 품목

## 수입규제품목(지역본부별) 정보 데이터 불러오기

In [5]:
# 불러오기
regl = pd.read_csv('regl_raw.csv')

In [6]:
# hscd 자리수 탐색
regl.sort_values('HSCD')

,HQURT_NAME,CMDLT_NAME,HSCD,HSCD_CN,REG_DT,REGL_CN,ISO_WD2_NAT_CD,REGL_STR_DE,REGL_END_DE,PROBE_TGT_NAT_NAME
6498,중동지역본부,유리 섬유 보강재(Glass Fiber Reinforcement Materials),31,"701911,701912,701919,7019,31,701990001000",2023-05-19,반덤핑(규제중),TR,20210324.0,20260323.0,태국
1917,서남아지역본부,실리콘 밀봉제 (Silicone Sealants),32,"32,35",2023-05-19,반덤핑(규제중),IN,20211221.0,20261220.0,중국
1916,서남아지역본부,실리콘 밀봉제 (Silicone Sealants),35,"32,35",2023-05-19,반덤핑(규제중),IN,20211221.0,20261220.0,중국
4384,북미지역본부,바이오 디젤(Biodiesel),38,38,2023-05-19,상계관세(규제중),US,20180104.0,20230103.0,"아르헨티나, 인도네시아"
11917,유럽지역본부,철강제품(Steel Products),72,"72,73",2023-05-19,세이프가드(규제중),GR,20210701.0,20240630.0,전세계
...,...,...,...,...,...,...,...,...,...,...
7492,중남미지역본부,원자 화기 플라스틱(ATOMIZERS PLASTIC),961610,96161001,2023-05-19,반덤핑(규제중),MX,20190422.0,20240421.0,중국
8617,중남미지역본부,보온병 (Vacuum flasks and other vacuum vessels wi...,961700,96170010,2023-05-19,반덤핑(규제중),AR,20011030.0,20240415.0,중국
236,동남아대양주지역본부,합금/비합금 철강제품(Semi-finished and certain finished...,981100,"720711,720719,720720,722490,721310,721420,7227...",2023-05-19,세이프가드(규제중),VN,20200322.0,20260321.0,전세계
349,동남아대양주지역본부,선 및 압연 제품 (Wire and rolled steel products),983910,"72139190,72139990,72171010,72171029,72299099,9...",2023-05-19,세이프가드(규제중),VN,20200322.0,20230321.0,전세계


## 데이터 전처리

- HSCD_CN: HS코드내용
- ISO_WD2_NAT_CD: ISO2자리국가코드(규제시행국)
- REGL_STR_DE: 규제시작일자
- REGL_END_DE: 규제종료일자
- PROBE_TGT_NAT_NAME: 조사대상국가명(대상국)

In [7]:
# 사용할 컬럼만 추출
regl = regl[['HSCD', 'HSCD_CN', 'ISO_WD2_NAT_CD', 'REGL_STR_DE', 'REGL_END_DE', 'PROBE_TGT_NAT_NAME']]

# 확인
regl

,HSCD,HSCD_CN,ISO_WD2_NAT_CD,REGL_STR_DE,REGL_END_DE,PROBE_TGT_NAT_NAME
0,720926,"720916,720917,720918,720926,720927,720928,7209...",VN,20201228.0,20251227.0,중국
1,250610,"2506,250610,6810,681019,681091,681099,6815,681599",US,20190711.0,20240710.0,중국
2,681019,"2506,250610,6810,681019,681091,681099,6815,681599",US,20190711.0,20240710.0,중국
3,283410,283410,US,20230103.0,20280102.0,인도
4,722619,"7225190000,7226191000,7226199000,7225508085,72...",US,20141006.0,20251222.0,"중국,대만"
...,...,...,...,...,...,...
12621,690723,"69072100,69072200,69072300,69073000,69074000",FI,20230211.0,20280211.0,"인도,터키"
12622,690723,"69072100,69072200,69072300,69073000,69074000",FR,20230211.0,20280211.0,"인도,터키"
12623,30482,"0301919011,0302118011,0303149011,0304429010,03...",AT,20210525.0,20260525.0,터키
12624,722699,"7210410020,7210490020,7210610020,7210690020,72...",CZ,20180208.0,20230208.0,중국


## 조사대상국가명이 한국인 것만 추출

In [8]:
# '조사대상국가명' == '한국' 추출
regl_k = regl.loc[(regl['PROBE_TGT_NAT_NAME'] == '한국')]

# 확인
regl_k

,HSCD,HSCD_CN,ISO_WD2_NAT_CD,REGL_STR_DE,REGL_END_DE,PROBE_TGT_NAT_NAME
278,720292,7202920000,US,20170515.0,20271128.0,한국
374,550320,550320,US,NaN,NaN,한국
507,390690,"3906905000,3609909000,3906100000",US,20221127.0,20271126.0,한국
508,390610,"3906905000,3609909000,3906100000",US,20221127.0,20271126.0,한국
509,360990,"3906905000,3609909000,3906100000",US,20221127.0,20271126.0,한국
...,...,...,...,...,...,...
11007,721934,"7219130030,7219130050,7219130070,7219130080,72...",US,20171003.0,20221002.0,한국
11008,721934,"7219130030,7219130050,7219130070,7219130080,72...",US,20171003.0,20221003.0,한국
11012,721933,"7219130030,7219130050,7219130070,7219130080,72...",US,20171003.0,20221002.0,한국
11013,721933,"7219130030,7219130050,7219130070,7219130080,72...",US,20171003.0,20221003.0,한국


In [9]:
# hscd_cn 제거
regl_k = regl_k.drop('HSCD_CN', axis=1)

# 한국으로 동일한 컬럼 PROBE_TGT_NAT_NAME 제거
regl_k = regl_k.drop('PROBE_TGT_NAT_NAME', axis=1)

# 확인
regl_k

,HSCD,ISO_WD2_NAT_CD,REGL_STR_DE,REGL_END_DE
278,720292,US,20170515.0,20271128.0
374,550320,US,NaN,NaN
507,390690,US,20221127.0,20271126.0
508,390610,US,20221127.0,20271126.0
509,360990,US,20221127.0,20271126.0
...,...,...,...,...
11007,721934,US,20171003.0,20221002.0
11008,721934,US,20171003.0,20221003.0
11012,721933,US,20171003.0,20221002.0
11013,721933,US,20171003.0,20221003.0


### 10개 국가 조회

In [10]:
# 네팔
regl_k.loc[regl_k['ISO_WD2_NAT_CD'] == 'NP']

,HSCD,ISO_WD2_NAT_CD,REGL_STR_DE,REGL_END_DE


In [11]:
#우루과이
regl_k.loc[regl_k['ISO_WD2_NAT_CD'] == 'UY']

,HSCD,ISO_WD2_NAT_CD,REGL_STR_DE,REGL_END_DE


In [12]:
# 가봉
regl_k.loc[regl_k['ISO_WD2_NAT_CD'] == 'GA']

,HSCD,ISO_WD2_NAT_CD,REGL_STR_DE,REGL_END_DE


In [13]:
# 볼리비아
regl_k.loc[regl_k['ISO_WD2_NAT_CD'] == 'BO']

,HSCD,ISO_WD2_NAT_CD,REGL_STR_DE,REGL_END_DE


In [14]:
# 베네주엘라
regl_k.loc[regl_k['ISO_WD2_NAT_CD'] == 'VE']

,HSCD,ISO_WD2_NAT_CD,REGL_STR_DE,REGL_END_DE


In [15]:
# 그리스
regl_k.loc[regl_k['ISO_WD2_NAT_CD'] == 'GR']

,HSCD,ISO_WD2_NAT_CD,REGL_STR_DE,REGL_END_DE
1142,481190,GR,20201020.0,20251021.0
1375,480990,GR,20170504.0,20280315.0
1385,481190,GR,20170504.0,20280315.0
1401,482390,GR,20170504.0,20280315.0
1411,481690,GR,20170504.0,20280315.0
1415,480990,GR,20201020.0,20251021.0
1425,481159,GR,20201020.0,20251021.0
4235,390690,GR,20220406.0,20270406.0


In [16]:
# 필리핀
regl_k.loc[regl_k['ISO_WD2_NAT_CD'] == 'PH']

,HSCD,ISO_WD2_NAT_CD,REGL_STR_DE,REGL_END_DE


In [17]:
# 호주
regl_k.loc[regl_k['ISO_WD2_NAT_CD'] == 'AU']

,HSCD,ISO_WD2_NAT_CD,REGL_STR_DE,REGL_END_DE


In [18]:
# 스페인
regl_k.loc[regl_k['ISO_WD2_NAT_CD'] == 'ES']

,HSCD,ISO_WD2_NAT_CD,REGL_STR_DE,REGL_END_DE
4659,481690,ES,20170504.0,20280315.0
4678,482390,ES,20170504.0,20280315.0
4699,481159,ES,20201020.0,20251021.0
4720,481190,ES,20201020.0,20251021.0
4875,480990,ES,20170504.0,20280315.0
4893,481190,ES,20170504.0,20280315.0
5006,480990,ES,20201020.0,20251021.0
5036,390690,ES,20220406.0,20270406.0


In [19]:
# 뉴질랜드
regl_k.loc[regl_k['ISO_WD2_NAT_CD'] == 'NZ']

,HSCD,ISO_WD2_NAT_CD,REGL_STR_DE,REGL_END_DE
8420,721220,NZ,20230101.0,20270329.0
8421,721090,NZ,20230101.0,20270329.0
8434,721049,NZ,20230101.0,20270329.0
8435,721061,NZ,20230101.0,20270329.0
8436,721069,NZ,20230101.0,20270329.0
8497,722699,NZ,20230101.0,20270329.0
8563,721230,NZ,20230101.0,20270329.0
8564,721250,NZ,20230101.0,20270329.0
8565,721260,NZ,20230101.0,20270329.0
8566,722511,NZ,20230101.0,20270329.0


# merge 준비

In [20]:
# merge를 위한 컬럼명 변경

regl_k.rename(columns={'HSCD':'품목', 'ISO_WD2_NAT_CD':'나라', 'REGL_STR_DE':'규제시작일자', 'REGL_END_DE':'규제종료일자'}, inplace=True)

# 확인
regl_k

,품목,나라,규제시작일자,규제종료일자
278,720292,US,20170515.0,20271128.0
374,550320,US,NaN,NaN
507,390690,US,20221127.0,20271126.0
508,390610,US,20221127.0,20271126.0
509,360990,US,20221127.0,20271126.0
...,...,...,...,...
11007,721934,US,20171003.0,20221002.0
11008,721934,US,20171003.0,20221003.0
11012,721933,US,20171003.0,20221002.0
11013,721933,US,20171003.0,20221003.0


In [21]:
# 저장
regl_k.to_csv('regl_k.csv', index=False, encoding="utf-8-sig")

## 비중, rca, 수입수출국 데이터 불러오기

In [22]:
# 불러오기
rca = pd.read_excel('예상비중_rca_수입수출국.xlsx')
rca

,나라,품목,예상 비중( 유사도 * 비중 / (유사도의 합) ),RCA,분자의 분자,분자의 분모,분모의 분자,분모의 분모,해당 나라 해당 품목 수입액,해당 나라 해당 품목 수출액
0,NP,870321,5.310011e-02,1.481797,1829455897,681729790292,41610258616,22976267162433,0,0
1,NP,870322,1.537141e-02,2.199953,6266563064,681729790292,96002635023,22976267162433,194870,0
2,NP,271019,8.294951e-03,2.624906,46318725022,681729790292,594716539591,22976267162433,740379,43551
3,NP,401120,7.738877e-03,1.173322,926614789,681729790292,26616388673,22976267162433,24495,0
4,NP,852589,1.679731e-03,3.349365,2716386822,681729790292,27333572062,22976267162433,0,303
...,...,...,...,...,...,...,...,...,...,...
295,NZ,731290,2.340200e-06,0.157012,1677039,681729790292,359979439,22976267162433,767430,110506
296,NZ,830242,1.339200e-06,0.182769,50652529,681729790292,9340383469,22976267162433,59095251,1105334
297,NZ,271020,9.318000e-07,0.006712,2864026,681729790292,14381110903,22976267162433,192705,20456
298,NZ,901730,1.663000e-07,0.264010,5945353,681729790292,758969952,22976267162433,2399901,25302


In [23]:
# RCA - 규제품목 merge
trade = pd.merge(rca, regl_k, on=['나라','품목'], how='left')
trade

,나라,품목,예상 비중( 유사도 * 비중 / (유사도의 합) ),RCA,분자의 분자,분자의 분모,분모의 분자,분모의 분모,해당 나라 해당 품목 수입액,해당 나라 해당 품목 수출액,규제시작일자,규제종료일자
0,NP,870321,5.310011e-02,1.481797,1829455897,681729790292,41610258616,22976267162433,0,0,NaN,NaN
1,NP,870322,1.537141e-02,2.199953,6266563064,681729790292,96002635023,22976267162433,194870,0,NaN,NaN
2,NP,271019,8.294951e-03,2.624906,46318725022,681729790292,594716539591,22976267162433,740379,43551,NaN,NaN
3,NP,401120,7.738877e-03,1.173322,926614789,681729790292,26616388673,22976267162433,24495,0,NaN,NaN
4,NP,852589,1.679731e-03,3.349365,2716386822,681729790292,27333572062,22976267162433,0,303,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
295,NZ,731290,2.340200e-06,0.157012,1677039,681729790292,359979439,22976267162433,767430,110506,NaN,NaN
296,NZ,830242,1.339200e-06,0.182769,50652529,681729790292,9340383469,22976267162433,59095251,1105334,NaN,NaN
297,NZ,271020,9.318000e-07,0.006712,2864026,681729790292,14381110903,22976267162433,192705,20456,NaN,NaN
298,NZ,901730,1.663000e-07,0.264010,5945353,681729790292,758969952,22976267162433,2399901,25302,NaN,NaN


In [24]:
# 저장
trade.to_csv('1차규제.csv', index=False, encoding="utf-8-sig")

# 2. 수입국, 수출국 여부
- 해당품목에 대한 수입국일 경우 추천

In [26]:
# 1차 규제 csv 불러오기
exim = pd.read_csv('1차규제.csv')
exim

,나라,품목,예상 비중( 유사도 * 비중 / (유사도의 합) ),RCA,분자의 분자,분자의 분모,분모의 분자,분모의 분모,해당 나라 해당 품목 수입액,해당 나라 해당 품목 수출액,규제시작일자,규제종료일자
0,NP,870321,5.310011e-02,1.481797,1829455897,681729790292,41610258616,22976267162433,0,0,NaN,NaN
1,NP,870322,1.537141e-02,2.199953,6266563064,681729790292,96002635023,22976267162433,194870,0,NaN,NaN
2,NP,271019,8.294951e-03,2.624906,46318725022,681729790292,594716539591,22976267162433,740379,43551,NaN,NaN
3,NP,401120,7.738877e-03,1.173322,926614789,681729790292,26616388673,22976267162433,24495,0,NaN,NaN
4,NP,852589,1.679731e-03,3.349365,2716386822,681729790292,27333572062,22976267162433,0,303,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
295,NZ,731290,2.340200e-06,0.157012,1677039,681729790292,359979439,22976267162433,767430,110506,NaN,NaN
296,NZ,830242,1.339200e-06,0.182769,50652529,681729790292,9340383469,22976267162433,59095251,1105334,NaN,NaN
297,NZ,271020,9.318000e-07,0.006712,2864026,681729790292,14381110903,22976267162433,192705,20456,NaN,NaN
298,NZ,901730,1.663000e-07,0.264010,5945353,681729790292,758969952,22976267162433,2399901,25302,NaN,NaN


In [27]:
# 해당 국가, 해당 품목이 수입국인 경우만 추출
exim_f = exim.loc[exim['해당 나라 해당 품목 수입액'] > exim['해당 나라 해당 품목 수출액']]
exim_f

,나라,품목,예상 비중( 유사도 * 비중 / (유사도의 합) ),RCA,분자의 분자,분자의 분모,분모의 분자,분모의 분모,해당 나라 해당 품목 수입액,해당 나라 해당 품목 수출액,규제시작일자,규제종료일자
1,NP,870322,1.537141e-02,2.199953,6266563064,681729790292,96002635023,22976267162433,194870,0,NaN,NaN
2,NP,271019,8.294951e-03,2.624906,46318725022,681729790292,594716539591,22976267162433,740379,43551,NaN,NaN
3,NP,401120,7.738877e-03,1.173322,926614789,681729790292,26616388673,22976267162433,24495,0,NaN,NaN
5,NP,848210,8.923570e-04,1.069231,463592712,681729790292,14612753495,22976267162433,75907,2809,NaN,NaN
8,NP,842139,2.256340e-04,1.151231,790447993,681729790292,23140788928,22976267162433,394930,4535,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
295,NZ,731290,2.340200e-06,0.157012,1677039,681729790292,359979439,22976267162433,767430,110506,NaN,NaN
296,NZ,830242,1.339200e-06,0.182769,50652529,681729790292,9340383469,22976267162433,59095251,1105334,NaN,NaN
297,NZ,271020,9.318000e-07,0.006712,2864026,681729790292,14381110903,22976267162433,192705,20456,NaN,NaN
298,NZ,901730,1.663000e-07,0.264010,5945353,681729790292,758969952,22976267162433,2399901,25302,NaN,NaN


In [28]:
# 품목 필터링 적용한 csv 파일 저장
exim_f.to_csv('품목필터링적용후.csv', index=False, encoding="utf-8-sig")